In [1]:

import numpy as np 
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

import folium # map rendering library

print('Libraries imported.')

Libraries imported.


In [5]:
ls

 Volume in drive C has no label.
 Volume Serial Number is B22B-E5E2

 Directory of C:\Users\user1\JUpyter\IBM Data Science

02/13/2020  12:57 PM    <DIR>          .
02/13/2020  12:57 PM    <DIR>          ..
02/13/2020  11:59 AM    <DIR>          .ipynb_checkpoints
02/09/2020  05:39 PM            39,780 capstone_note.ipynb
02/11/2020  10:35 PM            42,327 Copy of 2019 Q3 IGR Data.xlsx
06/08/2019  04:02 PM             4,680 DNA-Cluster Analysis.ipynb
05/02/2019  01:47 AM            72,629 FuelConsumption.csv
02/09/2020  04:51 PM             2,891 Geospatial_Coordinates.csv
05/14/2019  12:06 PM             2,750 Jupyter-Notebook.zip
02/13/2020  12:57 PM            10,600 Lagos_LGAs.xlsx
07/25/2019  02:45 PM           149,642 ML0101EN-Reg-Simple-Linear-Regression-Co2-py-v1.ipynb
02/12/2020  08:42 PM            10,731 States_Coordinates.xlsx
02/09/2020  04:47 PM             3,391 toronto_part1.csv
05/02/2019  02:14 PM            69,270 Untitled.ipynb
02/13/2020  11:39 AM            80

In [6]:
df = pd.read_excel("Lagos_LGAs.xlsx")

In [8]:
df.head()

,LGA,Census,Postal Code,Latitude,Longitude
0,Agege,459939,100,6.625256,3.311209
1,Ajeromi-Ifelodun,684105,102,6.455122,3.335946
2,Alimosho,1277714,100,6.584343,3.257631
3,Amuwo-Odofin,318166,102,6.447023,3.266280
4,Apapa,217362,101,6.445187,3.368373


In [15]:
df.columns

Index(['LGA', 'Census', 'Postal Code', 'Latitude', 'Longitude'], dtype='object')

In [11]:
# Lagos State Coordinates

Latitude = 6.4903288
Longitude = 3.3543284

In [17]:
# create map of Lagos State using latitude and longitude values
map_lagos = folium.Map(location=[Latitude, Longitude], zoom_start=10)

# add markers to map
for lat, lng, lga, census in zip(df['Latitude'], df['Longitude'], df['LGA'], df['Census']):
    label = '{}, {}'.format(lga, census)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_lagos)  
    
map_lagos

Exploring Surulere in our dataframe.¶

In [26]:
CLIENT_ID = '5JK4ZYOJCYJRJYUJKFFWHBOCNUYW2NEHUWSXL2UJLLN5VDIT' # your Foursquare ID
CLIENT_SECRET = '5ZDNNLRASPIYSW4XE3YYLAJ4CKPVVL2SKKU5LHH4ES0WZMIH' # your Foursquare Secret
VERSION = '20180604'
LIMIT = 100

print("Your credentails: "
      "CLIENT_ID :" ,CLIENT_ID,
      "CLIENT_SECRET:", CLIENT_SECRET)

Your credentails: CLIENT_ID : 5JK4ZYOJCYJRJYUJKFFWHBOCNUYW2NEHUWSXL2UJLLN5VDIT CLIENT_SECRET: 5ZDNNLRASPIYSW4XE3YYLAJ4CKPVVL2SKKU5LHH4ES0WZMIH


In [20]:
df.loc[19, 'LGA']

'Surulere'

In [25]:
# Get the latitude and longitude values for Surulere.
lungu_lat = df.loc[19, 'Latitude']
lungu_long = df.loc[19, 'Longitude']

lungu_name = df.loc[19, 'LGA']
radius=750
LIMIT=100

print('Latitude and Longitude values of {} are {}, {}'.format(lungu_name,
                                                               lungu_lat,
                                                               lungu_long))

Latitude and Longitude values of Surulere are 6.4903288, 3.3543284


### Get the top 100 venues that are located in Pelham Bay within the radius of 750 meters.¶


In [27]:
# First, create the GET request URL.

url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    lungu_lat, 
    lungu_long, 
    radius, 
    LIMIT)
url # display URL

'https://api.foursquare.com/v2/venues/explore?&client_id=5JK4ZYOJCYJRJYUJKFFWHBOCNUYW2NEHUWSXL2UJLLN5VDIT&client_secret=5ZDNNLRASPIYSW4XE3YYLAJ4CKPVVL2SKKU5LHH4ES0WZMIH&v=20180604&ll=6.4903288,3.3543284&radius=750&limit=100'

In [28]:
# Send the GET request and examie the results
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5e4540fa882fc7001bb0c61e'},
 'response': {'suggestedFilters': {'header': 'Tap to show:',
   'filters': [{'name': 'Open now', 'key': 'openNow'}]},
  'headerLocation': 'Surulere',
  'headerFullLocation': 'Surulere, Lagos',
  'headerLocationGranularity': 'neighborhood',
  'totalResults': 18,
  'suggestedBounds': {'ne': {'lat': 6.497078806750007,
    'lng': 3.361109262605372},
   'sw': {'lat': 6.483578793249993, 'lng': 3.347547537394628}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '50ccbc9ce4b0d07024dd14f4',
       'name': 'Film House Cinema',
       'location': {'address': 'Adeniran Ogunsanya',
        'lat': 6.490242,
        'lng': 3.3573713,
        'labeledLatLngs': [{'label': 'display',
          'lat': 6.490242,
          'lng

### All the information needed is in the items key. First write a function that extracts the category of venues.¶


In [29]:

def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [30]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

,name,categories,lat,lng
0,Film House Cinema,Multiplex,6.490242,3.357371
1,Filmhouse Cinema,Movie Theater,6.490105,3.357289
2,Leisure Mall,Shopping Mall,6.490129,3.357138
3,Mr. Price,Clothing Store,6.490194,3.356655
4,Shoprite,Shopping Mall,6.490594,3.357076


In [31]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

18 venues were returned by Foursquare.


In [33]:
# Plot out a map of the venues of Pelham Bay.
map_su = folium.Map(location=[Latitude, Longitude], zoom_start=10)

# add markers to map
for lat, lng, name, cat in zip(nearby_venues['lat'], nearby_venues['lng'], nearby_venues['name'], nearby_venues['categories']):
    label = '{}, {}'.format(name, cat)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_su)  
    
map_su

In [34]:
# Function to repeat the same process to all the LGAs in Lagos.
def getNearbyVenues(names, latitudes, longitudes, radius=750):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [36]:
# Run the above function on each neighborhood and create a new dataframe.
bxbk_venues = getNearbyVenues(names=df['LGA'],
                                 latitudes=df['Latitude'],
                                 longitudes=df['Longitude'])

Agege
Ajeromi-Ifelodun
Alimosho
Amuwo-Odofin
Apapa
Badagry
Epe
Eti-Osa
Ibeju-Lekki
Ifako-Ijaiye
Ikeja
Ikorodu
Kosofe
Lagos Island
Lagos Mainland
Mushin
Ojo
Oshodi-Isolo
Shomolu
Surulere


In [55]:
df_venues = bxbk_venues
df_venues.head()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Agege,6.625256,3.311209,Agbotikuyo,6.625004,3.313046,Bus Station
1,Agege,6.625256,3.311209,Ayige Gym,6.628867,3.305549,Gym
2,Amuwo-Odofin,6.447023,3.266280,Golden Tulip Festac Lagos,6.449725,3.269596,Hotel
3,Amuwo-Odofin,6.447023,3.266280,Destiny Suites,6.448511,3.270006,Hotel
4,Amuwo-Odofin,6.447023,3.266280,Grocery Bazaar,6.450635,3.267022,Department Store


In [42]:
df_venues.shape

(69, 7)

In [44]:
#Checkout how many venues were returned for each neighborhood
df_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Agege,2,2,2,2,2,2
Amuwo-Odofin,4,4,4,4,4,4
Apapa,8,8,8,8,8,8
Badagry,1,1,1,1,1,1
Eti-Osa,5,5,5,5,5,5
Ifako-Ijaiye,2,2,2,2,2,2
Ikeja,9,9,9,9,9,9
Ikorodu,4,4,4,4,4,4
Kosofe,1,1,1,1,1,1


## Analyze Each LGA.

In [45]:
# Get dummy variables.
df_onehot = pd.get_dummies(df_venues[['Venue Category']], prefix='', prefix_sep='')

# Add neighborhood column back to dataframe.
df_onehot['LGA'] = df_venues['Neighborhood']

# Move neighborhood column to the first column.
fixed_columns = [df_onehot.columns[-1]] + list(df_onehot.columns[:-1])
df_onehot = df_onehot[fixed_columns]

df_onehot.head()

,LGA,African Restaurant,Athletics & Sports,BBQ Joint,Bakery,Bar,Beer Garden,Bistro,Boat or Ferry,Boutique,Bowling Alley,Bus Station,Chinese Restaurant,Clothing Store,Convenience Store,Department Store,Electronics Store,Fast Food Restaurant,Flea Market,Food,Fried Chicken Joint,Furniture / Home Store,Gym,Gym / Fitness Center,Historic Site,Hotel,Ice Cream Shop,Jewelry Store,Lounge,Market,Mobile Phone Shop,Movie Theater,Multiplex,Pharmacy,Pizza Place,Plaza,Shopping Mall,Steakhouse
0,Agege,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,Agege,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,Amuwo-Odofin,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0
3,Amuwo-Odofin,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0
4,Amuwo-Odofin,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [46]:
df_onehot.shape

(69, 38)

### Group rows by neighborhood and by taking the mean of the frequency of occurance of each category

In [47]:

df_grouped = df_onehot.groupby('LGA').mean().reset_index()
df_grouped.head()

,LGA,African Restaurant,Athletics & Sports,BBQ Joint,Bakery,Bar,Beer Garden,Bistro,Boat or Ferry,Boutique,Bowling Alley,Bus Station,Chinese Restaurant,Clothing Store,Convenience Store,Department Store,Electronics Store,Fast Food Restaurant,Flea Market,Food,Fried Chicken Joint,Furniture / Home Store,Gym,Gym / Fitness Center,Historic Site,Hotel,Ice Cream Shop,Jewelry Store,Lounge,Market,Mobile Phone Shop,Movie Theater,Multiplex,Pharmacy,Pizza Place,Plaza,Shopping Mall,Steakhouse
0,Agege,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000,0.0,0.00,0.5,0.000,0.0,0.0,0.000,0.0,0.000,0.0,0.0,0.0,0.0,0.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000,0.0,0.000,0.00,0.0
1,Amuwo-Odofin,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000,0.0,0.25,0.0,0.000,0.0,0.0,0.250,0.0,0.000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000,0.0,0.000,0.00,0.0
2,Apapa,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.125,0.0,0.00,0.0,0.125,0.0,0.0,0.125,0.0,0.125,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.125,0.0,0.125,0.25,0.0
3,Badagry,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000,0.0,0.00,0.0,0.000,0.0,0.0,0.000,0.0,0.000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000,0.0,0.000,0.00,0.0
4,Eti-Osa,0.0,0.0,0.0,0.0,0.0,0.0,0.2,0.000,0.0,0.00,0.0,0.000,0.0,0.0,0.200,0.0,0.000,0.0,0.0,0.0,0.0,0.0,0.2,0.0,0.4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000,0.0,0.000,0.00,0.0


### Print out each neighborhood with the top 5 most common venues.¶

In [48]:

num_top_venues = 5

for hood in df_grouped['LGA']:
    print("----"+hood+"----")
    temp = df_grouped[df_grouped['LGA'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Agege----
                  venue  freq
0           Bus Station   0.5
1                   Gym   0.5
2    African Restaurant   0.0
3  Gym / Fitness Center   0.0
4         Historic Site   0.0


----Amuwo-Odofin----
                  venue  freq
0                 Hotel  0.50
1         Bowling Alley  0.25
2      Department Store  0.25
3                Market  0.00
4  Gym / Fitness Center  0.00


----Apapa----
                  venue  freq
0         Shopping Mall  0.25
1                 Plaza  0.12
2              Pharmacy  0.12
3         Boat or Ferry  0.12
4  Fast Food Restaurant  0.12


----Badagry----
                  venue  freq
0                 Hotel   1.0
1    African Restaurant   0.0
2   Fried Chicken Joint   0.0
3                   Gym   0.0
4  Gym / Fitness Center   0.0


----Eti-Osa----
                  venue  freq
0                 Hotel   0.4
1  Gym / Fitness Center   0.2
2      Department Store   0.2
3                Bistro   0.2
4    African Restaurant   0.0


----Ifako

### Put the most common venues into a dataframe

In [49]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [52]:
# Create the new dataframe and display the top 10 venues for each LGA.
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['LGA']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
lga_venues_sorted = pd.DataFrame(columns=columns)
lga_venues_sorted['LGA'] = df_grouped['LGA']

for ind in np.arange(df_grouped.shape[0]):
    lga_venues_sorted.iloc[ind, 1:] = return_most_common_venues(df_grouped.iloc[ind, :], num_top_venues)

lga_venues_sorted.head()


,LGA,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Agege,Gym,Bus Station,Steakhouse,Bowling Alley,Electronics Store,Department Store,Convenience Store,Clothing Store,Chinese Restaurant,Boutique
1,Amuwo-Odofin,Hotel,Bowling Alley,Department Store,Steakhouse,Electronics Store,Convenience Store,Clothing Store,Chinese Restaurant,Bus Station,Boutique
2,Apapa,Shopping Mall,Boat or Ferry,Plaza,Pharmacy,Department Store,Chinese Restaurant,Fast Food Restaurant,Bowling Alley,Convenience Store,Clothing Store
3,Badagry,Hotel,Steakhouse,Bowling Alley,Electronics Store,Department Store,Convenience Store,Clothing Store,Chinese Restaurant,Bus Station,Boutique
4,Eti-Osa,Hotel,Department Store,Gym / Fitness Center,Bistro,Steakhouse,Bowling Alley,Convenience Store,Clothing Store,Chinese Restaurant,Bus Station


In [53]:
lga_venues_sorted

,LGA,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Agege,Gym,Bus Station,Steakhouse,Bowling Alley,Electronics Store,Department Store,Convenience Store,Clothing Store,Chinese Restaurant,Boutique
1,Amuwo-Odofin,Hotel,Bowling Alley,Department Store,Steakhouse,Electronics Store,Convenience Store,Clothing Store,Chinese Restaurant,Bus Station,Boutique
2,Apapa,Shopping Mall,Boat or Ferry,Plaza,Pharmacy,Department Store,Chinese Restaurant,Fast Food Restaurant,Bowling Alley,Convenience Store,Clothing Store
3,Badagry,Hotel,Steakhouse,Bowling Alley,Electronics Store,Department Store,Convenience Store,Clothing Store,Chinese Restaurant,Bus Station,Boutique
4,Eti-Osa,Hotel,Department Store,Gym / Fitness Center,Bistro,Steakhouse,Bowling Alley,Convenience Store,Clothing Store,Chinese Restaurant,Bus Station
5,Ifako-Ijaiye,Bar,Beer Garden,Steakhouse,Bowling Alley,Electronics Store,Department Store,Convenience Store,Clothing Store,Chinese Restaurant,Bus Station
6,Ikeja,Fast Food Restaurant,Electronics Store,Bakery,Hotel,African Restaurant,BBQ Joint,Boutique,Bar,Beer Garden,Bistro
7,Ikorodu,Electronics Store,Bar,Historic Site,Bus Station,Steakhouse,Bowling Alley,Department Store,Convenience Store,Clothing Store,Chinese Restaurant
8,Kosofe,Steakhouse,Flea Market,Electronics Store,Department Store,Convenience Store,Clothing Store,Chinese Restaurant,Bus Station,Bowling Alley,Boutique
9,Lagos Mainland,Lounge,BBQ Joint,Boutique,Electronics Store,Department Store,Convenience Store,Clothing Store,Chinese Restaurant,Bus Station,Bowling Alley


## Cluster Neighborhoods

In [57]:

# Set the number of clusters
kclusters = 5

df_clustering = df_grouped.drop('LGA', 1)

# Run the k-means clustering.
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(df_clustering)

# Check the cluster labels generated for each row in the dataframe.
kmeans.labels_[0:10]

array([0, 1, 2, 1, 1, 2, 2, 2, 3, 2])

### Create a new dataframe that includes the clusters as well as the top 10 venues for each neighborhood.

In [59]:
# add clustering labels
lga_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

df_merged = df

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
df_merged = df_merged.join(lga_venues_sorted.set_index('LGA'), on='LGA')

df_merged.head()


,LGA,Census,Postal Code,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Agege,459939,100,6.625256,3.311209,0.0,Gym,Bus Station,Steakhouse,Bowling Alley,Electronics Store,Department Store,Convenience Store,Clothing Store,Chinese Restaurant,Boutique
1,Ajeromi-Ifelodun,684105,102,6.455122,3.335946,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Alimosho,1277714,100,6.584343,3.257631,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Amuwo-Odofin,318166,102,6.447023,3.266280,1.0,Hotel,Bowling Alley,Department Store,Steakhouse,Electronics Store,Convenience Store,Clothing Store,Chinese Restaurant,Bus Station,Boutique
4,Apapa,217362,101,6.445187,3.368373,2.0,Shopping Mall,Boat or Ferry,Plaza,Pharmacy,Department Store,Chinese Restaurant,Fast Food Restaurant,Bowling Alley,Convenience Store,Clothing Store


In [121]:
df_merged.shape

(20, 16)

In [122]:
df_merged.dropna(axis=0, inplace=True)

In [123]:
df_merged.shape

(15, 16)

In [124]:
#df = df.astype({"a": int, "b": complex})

df_merged = df_merged.astype({"Cluster Labels": int})

### Finally visualize the resulting clusters on the map.¶


In [126]:
# create map
map_clusters = folium.Map(location=[Latitude, Longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(df_merged['Latitude'], df_merged['Longitude'], df_merged['LGA'], df_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters


In [65]:
folium.CircleMarker?

In [87]:
#Cluster 1
df_merged.loc[df_merged['Cluster Labels'] == 0, df_merged.columns[[1] + list(range(5, df_merged.shape[1]))]]

,Census,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,459939,0.0,Gym,Bus Station,Steakhouse,Bowling Alley,Electronics Store,Department Store,Convenience Store,Clothing Store,Chinese Restaurant,Boutique
17,621509,0.0,Bus Station,Market,Steakhouse,Boutique,Electronics Store,Department Store,Convenience Store,Clothing Store,Chinese Restaurant,Bowling Alley


In [88]:
#Cluster 2
df_merged.loc[df_merged['Cluster Labels'] == 1, df_merged.columns[[1] + list(range(5, df_merged.shape[1]))]]

,Census,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
3,318166,1.0,Hotel,Bowling Alley,Department Store,Steakhouse,Electronics Store,Convenience Store,Clothing Store,Chinese Restaurant,Bus Station,Boutique
5,241093,1.0,Hotel,Steakhouse,Bowling Alley,Electronics Store,Department Store,Convenience Store,Clothing Store,Chinese Restaurant,Bus Station,Boutique
7,287785,1.0,Hotel,Department Store,Gym / Fitness Center,Bistro,Steakhouse,Bowling Alley,Convenience Store,Clothing Store,Chinese Restaurant,Bus Station


In [92]:
#Cluster 3
df_merged.loc[df_merged['Cluster Labels'] == 2, df_merged.columns[[1] + list(range(5, df_merged.shape[1]))]]

,Census,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
4,217362,2.0,Shopping Mall,Boat or Ferry,Plaza,Pharmacy,Department Store,Chinese Restaurant,Fast Food Restaurant,Bowling Alley,Convenience Store,Clothing Store
9,427878,2.0,Bar,Beer Garden,Steakhouse,Bowling Alley,Electronics Store,Department Store,Convenience Store,Clothing Store,Chinese Restaurant,Bus Station
10,313196,2.0,Fast Food Restaurant,Electronics Store,Bakery,Hotel,African Restaurant,BBQ Joint,Boutique,Bar,Beer Garden,Bistro
11,535619,2.0,Electronics Store,Bar,Historic Site,Bus Station,Steakhouse,Bowling Alley,Department Store,Convenience Store,Clothing Store,Chinese Restaurant
14,317720,2.0,Lounge,BBQ Joint,Boutique,Electronics Store,Department Store,Convenience Store,Clothing Store,Chinese Restaurant,Bus Station,Bowling Alley
15,633009,2.0,Department Store,Bar,Market,Flea Market,Bus Station,Steakhouse,Bowling Alley,Electronics Store,Convenience Store,Clothing Store
18,402673,2.0,African Restaurant,Bar,Convenience Store,Fast Food Restaurant,Bakery,BBQ Joint,Beer Garden,Bistro,Boat or Ferry,Flea Market
19,503975,2.0,Shopping Mall,Fast Food Restaurant,Clothing Store,Furniture / Home Store,Athletics & Sports,Pizza Place,Multiplex,Movie Theater,Mobile Phone Shop,Jewelry Store


In [93]:
#Cluster 4
df_merged.loc[df_merged['Cluster Labels'] == 3, df_merged.columns[[1] + list(range(5, df_merged.shape[1]))]]

,Census,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
12,665393,3.0,Steakhouse,Flea Market,Electronics Store,Department Store,Convenience Store,Clothing Store,Chinese Restaurant,Bus Station,Bowling Alley,Boutique


In [94]:
#Cluster 5
df_merged.loc[df_merged['Cluster Labels'] == 4, df_merged.columns[[1] + list(range(5, df_merged.shape[1]))]]

,Census,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
16,598071,4.0,Food,Flea Market,Electronics Store,Department Store,Convenience Store,Clothing Store,Chinese Restaurant,Bus Station,Bowling Alley,Boutique
